In [39]:
import numpy as np
import models
import utility
import torch
from torch import optim
from torch import nn    
from torch.autograd import Function
from sklearn.model_selection import train_test_split
torch.manual_seed(0)

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs=5000
batch_size=256
hid_y=100
BCE=nn.BCELoss()
MSE=nn.MSELoss(reduction='mean')

In [42]:
total_loss=[]
tr_pehe=[]
val_pehe=[]
reg_loss_tr=[]
classT_loss_tr=[]
classW_loss_tr=[]
wasser_loss_tr=[]
rc_loss=[]
OR_reg_li=[]

val_reg_li=[]
val_tloss_li=[]
val_wasser_li=[]
val_rec_li=[]

def train_CFR(x_data,y_data):
   
    torch.manual_seed(0)
    #device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    best_val_loss = np.inf
    min_val_pehe=np.inf
    epochs_without_improvement = 0
    #threshold=0.00001
    patience=400
    #x_data,y_data=get_data('train',1)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.20)
    # write input_feature instead of 25
    net=models.TarNet(input_features,lat_dim,.1)
    #wnet=Wclassifier(lat_dim,.1)
    tnet=models.Tclassifier(lat_dim*2,.1)
    rnet=models.Regressors(lat_dim*2,hid_y,.1)
    dnet=models.Decoder(lat_dim*4,input_features,.1)
    net.to(device)
    tnet.to(device)
    rnet.to(device)
    dnet.to(device)
    
    opt_net = torch.optim.Adam(net.parameters(), lr=0.00002,weight_decay=1e-3)#1e-4
    opt_nett = torch.optim.Adam(tnet.parameters(), lr=0.00002, weight_decay=1e-3)
    opt_netr = torch.optim.Adam(rnet.parameters(), lr=0.00002,weight_decay=1e-3)
    opt_netd = torch.optim.Adam(dnet.parameters(), lr=0.00002,weight_decay=1e-3) #,weight_decay=1e-4
    
   
   

    for ep in range(1,epochs+1 ):
 
        train_dataloader_sr, train_dataloader_tr=utility.get_dataloader(X_train,y_train,batch_size)
        tot_los=0
        reg_los=0
        t_los=0
        w_los=0
        wa_los=0
        rc_los=0
        rereg_los=0
        cnt=0
 
        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):
            
            xs,ys=train_source_data
            xt,yt=train_target_data
            # Replace 30 with :
            xs_train=xs[:,5:].to(device)
            xt_train=xt[:,5:].to(device)
            ys.to(device)
            yt.to(device)
            train_x=torch.cat((xs_train,xt_train),0)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1).to(device)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1).to(device)
            concat_true=torch.cat((train_y,true_t),1)
            prop_t1=(xt_train.shape[0]/train_x.shape[0])
            phi_gamma,phi_delta,phi_upsilon,phi_irr=net(train_x)
            phi_all=torch.cat((phi_gamma,phi_delta,phi_upsilon,phi_irr), 1)
            del_ups=torch.cat((phi_delta, phi_upsilon), 1)
            gam_del=torch.cat((phi_gamma,phi_delta), 1)
            
            concat_pred=rnet(del_ups)
            #w_f=wnet(phi_delta)
            w_t=tnet(gam_del)
            decoded_space=dnet(phi_all)
            
            predicted_y=torch.unsqueeze(torch.where(true_t.squeeze() == 0, concat_pred[:,0], concat_pred[:,1]),dim=1)
            opt_net.zero_grad()
           
            opt_nett.zero_grad()
            opt_netr.zero_grad()
            opt_netd.zero_grad()
        
            
           
            
            
         
            Rloss=MSE(predicted_y,train_y)
           
            Reconstruction_loss=(MSE(decoded_space[:,0:6],train_x[:,0:6])+BCE(decoded_space[:,6:25],train_x[:,6:25])+MSE(decoded_space[:,25:],train_x[:,25:]))
            
            Wassloss,dist=utility.wasserstein(phi_upsilon,true_t)
           
            Tloss=BCE(w_t, true_t)
            cnt=cnt+1
            
            
        
            w1,w2,w3,w4=utility.cal_weightmatr(net)
            OR=utility.cal_RO(w1,w2,w3,w4)
            OR_re=utility.OR_reg(w1,w2,w3,w4)
            OR_loss=OR+OR_re
           
            combined_loss=Rloss+(Wassloss)+(Tloss)+(5*OR)+(OR_re)+(Reconstruction_loss)
            
            combined_loss.backward()
            tot_los=tot_los+combined_loss.item()
            reg_los=reg_los+Rloss.item()
            t_los=t_los+Tloss.item()
            w_los=w_los+OR.item()
            wa_los=wa_los+Wassloss.item()
            rc_los=rc_los+Reconstruction_loss.item()
            rereg_los=rereg_los+OR_re.item()
            # optimize
            
            
            opt_net.step()
           
            opt_nett.step()
            opt_netr.step()
            opt_netd.step()
            
            

      
        val_PEHE=utility.cal_pehe_nn(X_val, y_val,rnet,net)
        tr_PEHE=utility.cal_pehe(X_train,y_train,rnet,net)


         
        # Evaluation of validation
        
        X_val_t=X_val.to_numpy()
        X_val_t=torch.from_numpy(X_val_t.astype(np.float32)).to(device)
        
        y_val_t=y_val.to_numpy()
        y_val_t=torch.from_numpy(y_val_t.astype(np.float32)).to(device)
        
       
        
        if (ep>1000):
            
            if (val_PEHE<min_val_pehe):
                min_val_pehe=val_PEHE
                torch.save(net.state_dict(), 'best_encoder.pth')
                torch.save(rnet.state_dict(), 'best_regressor.pth')
                #print(ep)


            if (val_PEHE < best_val_loss):
                best_val_loss = val_PEHE
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            """
            if (epochs_without_improvement >= patience): #
                #print("Early stopping: Validation loss did not improve for {patience} epochs.")
                break

            """
        
        reg_loss_tr.append(reg_los)
        classT_loss_tr.append(t_los)
        classW_loss_tr.append(w_los)
        wasser_loss_tr.append(wa_los)
        total_loss.append(tot_los)
        tr_pehe.append(tr_PEHE)
        val_pehe.append(val_PEHE)
        rc_loss.append(rc_los)
        OR_reg_li.append(rereg_los)

    return rnet,net

In [43]:
lat_dim_li=[5]
dummy_list=[5]# 0,5,10,15,20,25
for ldi,ld in enumerate(lat_dim_li):
    lat_dim=ld
    print('latent dimensions', lat_dim)
    for di,d in enumerate(dummy_list):
    
        
        pehe_l=[]
        dummies=d
        input_features=25+dummies
        for i in range(1,2):
            clean()
            x_data_tr,y_data_tr=utility.get_data('train',i)
            x_data_n=utility.add_dummy_features_shuffle(x_data_tr,dummies)
            Regressor,Encoder=train_CFR(x_data_n,y_data_tr)
            
            
            #_______________loading best models
            Enc=models.TarNet(input_features,lat_dim,.1)
            Enc.to(device)
            Reg=models.Regressors(lat_dim*2,hid_y,.1)
            Reg.to(device)
            Enc.load_state_dict(torch.load('best_encoder.pth'))
            Reg.load_state_dict(torch.load('best_regressor.pth'))
            #____________________________
    
            data,y=utility.get_data('test',i)
            data_n=utility.add_dummy_features_shuffle(data,dummies)
            pehe_l.append(utility.cal_pehe(data_n,y,Reg,Enc))
            print('files completed: ',i)
    
    
        print('Dummies shuffling: ',d)
        print('PEHE mean', np.mean(pehe_l))
        print('PEHE std', np.std(pehe_l))
        print('*************************************************')
    print('___________________________Next dimension_______________________________')

latent dimensions 5
files completed:  1
Dummies shuffling:  5
PEHE mean 0.8146402835845947
PEHE std 0.0
*************************************************
___________________________Next dimension_______________________________


In [ ]:
"""
def clean():
    total_loss.clear()
    reg_loss_tr.clear()
    classT_loss_tr.clear()
    classW_loss_tr.clear()
    wasser_loss_tr.clear()
"""